# Libraries

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action="ignore")
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Pre-processing

In [2]:
#!python ../Preprocessing.py
claims = pd.read_csv("./claims.csv", low_memory=False)

In [3]:
claims

,Unnamed: 0,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,IsOutpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,ClaimDuration,NoPhy,AllPhy,SameAttOper,AdmisDuration,AgeAtClm,TotalRev,ClmYear,ClmMonth,ClmWeek,InsCovRatio,RevPerDay,Chronic_Sum,code_all_nan
0,0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1943-01-01,NaN,1,1,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,1,6,False,False,False,6.0,66,27068.0,2009,4,15,0.960544,3866.857143,7,False
1,1,BENE16973,CLM565430,2009-09-06,2009-09-06,PRV55912,50,PHY365867,PHY327147,NaN,NaN,NaN,0.0,NaN,NaN,V7183,53081,78959,4280,E8788,78079,79902,25002,71848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1932-05-01,NaN,1,1,0,39,310,12,12,1,1,1,0,0,0,0,1,0,0,0,24000,2136,450,200,1,0,False,True,False,NaN,77,50.0,2009,9,36,1.000000,50.000000,4,False
2,2,BENE17521,CLM34721,2009-01-20,2009-02-01,PRV55912,19000,PHY349293,PHY370861,PHY363291,2009-01-20,45340,1068.0,2009-02-01,987,4240,2639,2948,40390,45821,28489,5854,2753,E9305,NaN,7769.0,5849.0,NaN,NaN,NaN,NaN,0,1913-12-01,NaN,0,1,0,39,230,12,12,1,1,1,0,1,0,0,1,0,0,0,19000,1068,100,20,1,12,False,True,False,12.0,95,20068.0,2009,1,4,0.946781,1543.692308,5,False
3,3,BENE21718,CLM72336,2009-10-17,2009-11-04,PRV55912,17000,PHY334706,PHY334706,NaN,2009-10-17,V5789,1068.0,2009-11-04,941,V5789,4168,73313,7812,7993,78830,72273,43812,4019,NaN,9338.0,NaN,NaN,NaN,NaN,NaN,0,1922-10-01,NaN,1,1,0,39,600,12,12,0,0,0,0,0,0,0,1,1,0,0,17000,1068,1050,540,1,18,False,True,True,18.0,87,18068.0,2009,10,42,0.940890,950.947368,2,False
4,4,BENE22934,CLM73394,2009-10-25,2009-10-29,PRV55912,13000,PHY390614,PHY323689,PHY363291,2009-10-25,71946,1068.0,2009-10-29,506,71535,71960,4019,V1202,4240,2449,2768,NaN,NaN,NaN,8154.0,NaN,NaN,NaN,NaN,NaN,0,1930-07-01,NaN,0,1,0,39,280,12,12,0,1,0,0,1,0,1,1,1,1,0,27000,2136,450,160,1,4,False,True,False,4.0,79,14068.0,2009,10,43,0.924083,2813.600000,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,558206,BENE154147,CLM394122,2009-06-02,2009-06-04,PRV54050,500,PHY317497,NaN,PHY317497,NaN,NaN,0.0,NaN,NaN,71698,71941,7862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1925-01-01,NaN,0,1,0,23,400,12,12,0,0,0,0,0,0,0,1,1,0,0,0,0,890,120,0,2,False,False,False,NaN,84,500.0,2009,6,23,1.000000,166.666667,2,False
558207,558207,BENE154687,CLM184358,2009-02-08,2009-02-08,PRV54302,3300,PHY376238,PHY376238,NaN,NaN,99639,0.0,NaN,NaN,99661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1927-05-01,NaN,1,2,0,25,150,12,12,1,0,1,0,0,0,1,1,0,1,0,0,0,4400,220,0,0,False,True,True,NaN,82,3300.0,2009,2,6,1.000000,3300.000000,5,False
558208,558208,BENE157378,CLM460770,2009-07-09,2009-07-29,PRV51577,2100,PHY338096,NaN,NaN,NaN,NaN,0.0,NaN,NaN,5854,7907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Feature Engineering

- Mean claim cost per physician

In [4]:
claims['ClaimCost'] = claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid

avgClaimCostPerPhysician = claims.groupby(['Provider','AttendingPhysician'])['ClaimCost'].mean()
avgClaimCostPerPhysician = avgClaimCostPerPhysician.groupby('Provider').mean()
avgClaimCostPerPhysician

Provider
PRV51001    5857.250000
PRV51003     893.959840
PRV51004     435.616855
PRV51005     250.440584
PRV51007    1786.738315
               ...     
PRV57759     384.642857
PRV57760     243.872549
PRV57761     260.483193
PRV57762    1900.000000
PRV57763     372.881356
Name: ClaimCost, Length: 5410, dtype: float64

In [5]:
claims['ClaimCost'] = claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid

avgClaimCostPerPhysician = claims.groupby(['Provider','OperatingPhysician'])['ClaimCost'].mean()
avgClaimCostPerPhysician = avgClaimCostPerPhysician.groupby('Provider').mean()
avgClaimCostPerPhysician

Provider
PRV51001     9827.200000
PRV51003     3820.703187
PRV51004      268.728070
PRV51005      370.014898
PRV51007     1804.266667
                ...     
PRV57757    18151.333333
PRV57759      300.000000
PRV57760      187.000000
PRV57761      249.766667
PRV57763      765.789474
Name: ClaimCost, Length: 4853, dtype: float64

In [6]:
claims['ClaimCost'] = claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid

avgClaimCostPerPhysician = claims.groupby(['Provider','OtherPhysician'])['ClaimCost'].mean()
avgClaimCostPerPhysician = avgClaimCostPerPhysician.groupby('Provider').mean()
avgClaimCostPerPhysician

Provider
PRV51001     5574.125000
PRV51003      395.000000
PRV51004      433.933150
PRV51005      256.092600
PRV51007      130.682540
                ...     
PRV57757    39068.000000
PRV57759      240.833333
PRV57760      146.785714
PRV57761      282.592593
PRV57763      325.918367
Name: ClaimCost, Length: 4878, dtype: float64

- Mean states per physician

In [21]:
avgDistinctStatePerPhysician_Attending = claims.groupby(['Provider','AttendingPhysician']
                                                       )['State'].nunique()
avgDistinctStatePerPhysician_Attending = avgDistinctStatePerPhysician_Attending.groupby(
    'Provider').mean().reset_index().rename(columns={'State':'Mean_StatePerAttPhys'})
avgDistinctStatePerPhysician_Attending

,Provider,Mean_StatePerAttPhys
0,PRV51001,1.000000
1,PRV51003,1.295455
2,PRV51004,2.131579
3,PRV51005,2.333333
4,PRV51007,1.300000
...,...,...
5405,PRV57759,1.000000
5406,PRV57760,1.000000
5407,PRV57761,1.000000
5408,PRV57762,1.000000


In [22]:
avgDistinctStatePerPhysician_Operating = claims.groupby(['Provider','OperatingPhysician'])['State'].nunique()
avgDistinctStatePerPhysician_Operating = avgDistinctStatePerPhysician_Operating.groupby(
    'Provider').mean().reset_index().rename(columns={'State':'Mean_StatePerOperPhys'})
avgDistinctStatePerPhysician_Operating

,Provider,Mean_StatePerOperPhys
0,PRV51001,1.000000
1,PRV51003,1.571429
2,PRV51004,1.263158
3,PRV51005,1.500000
4,PRV51007,1.000000
...,...,...
4848,PRV57757,1.000000
4849,PRV57759,1.000000
4850,PRV57760,1.000000
4851,PRV57761,1.000000


In [23]:
avgDistinctStatePerPhysician_Other = claims.groupby(['Provider','OtherPhysician'])['State'].nunique()
avgDistinctStatePerPhysician_Other = avgDistinctStatePerPhysician_Other.groupby(
    'Provider').mean().reset_index().rename(columns={'State':'Mean_StatePerOthPhys'})
avgDistinctStatePerPhysician_Other

,Provider,Mean_StatePerOthPhys
0,PRV51001,1.000000
1,PRV51003,1.090909
2,PRV51004,1.692308
3,PRV51005,2.250000
4,PRV51007,1.000000
...,...,...
4873,PRV57757,1.000000
4874,PRV57759,1.000000
4875,PRV57760,1.000000
4876,PRV57761,1.000000


- Mean states per patient

In [10]:
avgDistinctStatePerPatient_Inpatient = claims[claims['IsOutpatient']==0].groupby(
    ['Provider','BeneID'])['State'].nunique()
avgDistinctStatePerPatient_Inpatient = avgDistinctStatePerPatient_Inpatient.groupby('Provider').mean()
avgDistinctStatePerPatient_Inpatient.value_counts()

1    2092
Name: State, dtype: int64

In [11]:
avgDistinctStatePerPatient_Inpatient = claims[claims['IsOutpatient']==1].groupby(
    ['Provider','BeneID'])['State'].nunique()
avgDistinctStatePerPatient_Inpatient = avgDistinctStatePerPatient_Inpatient.groupby('Provider').mean()
avgDistinctStatePerPatient_Inpatient.value_counts()

1    5012
Name: State, dtype: int64

- Mean IP/OP per physician

In [27]:
avgDistinctInpatientsPerPhysician_Attending = claims[claims['IsOutpatient']==0].groupby(
    ['Provider','AttendingPhysician'])['BeneID'].nunique()
avgDistinctInpatientsPerPhysician_Attending = avgDistinctInpatientsPerPhysician_Attending.groupby(
    'Provider').mean().reset_index().rename(columns={'BeneID':'IP_Mean_AttPhys'})
avgDistinctInpatientsPerPhysician_Attending

,Provider,IP_Mean_AttPhys
0,PRV51001,1.25
1,PRV51003,29.50
2,PRV51007,1.50
3,PRV51008,1.00
4,PRV51011,1.00
...,...,...
2084,PRV57719,1.00
2085,PRV57728,1.00
2086,PRV57729,1.10
2087,PRV57732,5.50


In [28]:
avgDistinctInpatientsPerPhysician_Operating = claims[claims['IsOutpatient']==0].groupby(
    ['Provider','OperatingPhysician'])['BeneID'].nunique()
avgDistinctInpatientsPerPhysician_Operating = avgDistinctInpatientsPerPhysician_Operating.groupby(
    'Provider').mean().reset_index().rename(columns={'BeneID':'IP_Mean_OperPhys'})
avgDistinctInpatientsPerPhysician_Operating

,Provider,IP_Mean_OperPhys
0,PRV51001,1.000000
1,PRV51003,20.000000
2,PRV51007,1.000000
3,PRV51008,1.000000
4,PRV51021,2.285714
...,...,...
1954,PRV57717,1.333333
1955,PRV57719,1.000000
1956,PRV57729,1.200000
1957,PRV57732,8.000000


In [29]:
avgDistinctInpatientsPerPhysician_Other = claims[claims['IsOutpatient']==0].groupby(
    ['Provider','OtherPhysician'])['BeneID'].nunique()
avgDistinctInpatientsPerPhysician_Other = avgDistinctInpatientsPerPhysician_Other.groupby(
    'Provider').mean().reset_index().rename(columns={'BeneID':'IP_Mean_OthPhys'})
avgDistinctInpatientsPerPhysician_Other

,Provider,IP_Mean_OthPhys
0,PRV51001,1.0
1,PRV51021,4.0
2,PRV51023,1.0
3,PRV51024,2.0
4,PRV51038,2.0
...,...,...
1152,PRV57719,1.0
1153,PRV57728,1.0
1154,PRV57729,1.0
1155,PRV57732,3.0


In [30]:
avgDistinctOutpatientsPerPhysician_Attending = claims[claims['IsOutpatient']==1].groupby(
    ['Provider','AttendingPhysician'])['BeneID'].nunique()
avgDistinctOutpatientsPerPhysician_Attending = avgDistinctOutpatientsPerPhysician_Attending.groupby(
    'Provider').mean().reset_index().rename(columns={'BeneID':'OP_Mean_AttPhys'})
avgDistinctOutpatientsPerPhysician_Attending

,Provider,OP_Mean_AttPhys
0,PRV51001,2.000000
1,PRV51003,1.590909
2,PRV51004,3.921053
3,PRV51005,147.833333
4,PRV51007,8.000000
...,...,...
5006,PRV57759,24.000000
5007,PRV57760,4.333333
5008,PRV57761,34.500000
5009,PRV57762,1.000000


In [31]:
avgDistinctOutpatientsPerPhysician_Operating = claims[claims['IsOutpatient']==1].groupby(
    ['Provider','OperatingPhysician'])['BeneID'].nunique()
avgDistinctOutpatientsPerPhysician_Operating = avgDistinctOutpatientsPerPhysician_Operating.groupby(
    'Provider').mean().reset_index().rename(columns={'BeneID':'OP_Mean_OperPhys'})
avgDistinctOutpatientsPerPhysician_Operating

,Provider,OP_Mean_OperPhys
0,PRV51001,1.000000
1,PRV51003,1.000000
2,PRV51004,1.421053
3,PRV51005,33.833333
4,PRV51007,2.750000
...,...,...
4374,PRV57756,2.500000
4375,PRV57759,1.000000
4376,PRV57760,2.500000
4377,PRV57761,2.800000


In [32]:
avgDistinctOutpatientsPerPhysician_Other = claims[claims['IsOutpatient']==1].groupby(
    ['Provider','OtherPhysician'])['BeneID'].nunique()
avgDistinctOutpatientsPerPhysician_Other = avgDistinctOutpatientsPerPhysician_Other.groupby(
    'Provider').mean().reset_index().rename(columns={'BeneID':'OP_Mean_OthPhys'})
avgDistinctOutpatientsPerPhysician_Other

,Provider,OP_Mean_OthPhys
0,PRV51001,1.285714
1,PRV51003,1.136364
2,PRV51004,2.423077
3,PRV51005,102.750000
4,PRV51007,3.571429
...,...,...
4661,PRV57756,5.500000
4662,PRV57759,11.000000
4663,PRV57760,2.000000
4664,PRV57761,17.000000


# Make new features into csv

In [33]:
providers = pd.DataFrame({'Provider': claims['Provider'].unique()})
providers

,Provider
0,PRV55912
1,PRV55907
2,PRV56046
3,PRV52405
4,PRV52314
...,...
5405,PRV54050
5406,PRV54302
5407,PRV51577
5408,PRV53083


In [34]:
providers = providers.join(avgDistinctStatePerPhysician_Attending.set_index('Provider'), on='Provider'
                    ).join(avgDistinctStatePerPhysician_Operating.set_index('Provider'), on='Provider'
                    ).join(avgDistinctStatePerPhysician_Other.set_index('Provider'), on='Provider'
                    ).join(avgDistinctInpatientsPerPhysician_Attending.set_index('Provider'), on='Provider'
                    ).join(avgDistinctInpatientsPerPhysician_Operating.set_index('Provider'), on='Provider'
                    ).join(avgDistinctInpatientsPerPhysician_Other.set_index('Provider'), on='Provider'
                    ).join(avgDistinctOutpatientsPerPhysician_Attending.set_index('Provider'), on='Provider'
                    ).join(avgDistinctOutpatientsPerPhysician_Operating.set_index('Provider'), on='Provider'
                    ).join(avgDistinctOutpatientsPerPhysician_Other.set_index('Provider'), on='Provider'
                          ).fillna(0)

providers

,Provider,Mean_StatePerAttPhys,Mean_StatePerOperPhys,Mean_StatePerOthPhys,IP_Mean_AttPhys,IP_Mean_OperPhys,IP_Mean_OthPhys,OP_Mean_AttPhys,OP_Mean_OperPhys,OP_Mean_OthPhys
0,PRV55912,1.102941,1.000000,1.083333,1.621622,1.480000,1.125,1.277778,1.00000,1.176471
1,PRV55907,1.580247,1.238095,1.147059,1.714286,1.590909,1.000,3.433962,1.26087,2.000000
2,PRV56046,1.000000,1.000000,1.000000,6.500000,5.000000,3.000,2.000000,1.00000,0.000000
3,PRV52405,1.529412,1.500000,1.222222,3.142857,2.000000,2.000,4.769231,2.00000,3.428571
4,PRV52314,1.568421,1.145833,1.277778,1.379310,1.090909,1.000,2.316456,1.12500,1.529412
...,...,...,...,...,...,...,...,...,...,...
5405,PRV54050,1.000000,0.000000,1.000000,0.000000,0.000000,0.000,1.000000,0.00000,1.000000
5406,PRV54302,1.000000,1.000000,0.000000,0.000000,0.000000,0.000,1.000000,1.00000,0.000000
5407,PRV51577,1.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,0.00000,0.000000
5408,PRV53083,1.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,0.00000,0.000000


In [35]:
providers.to_csv(r'./data/providers_second_iteration.csv', index = False)